![title](img/decisionmaking.png)

# Modeling decision making

Begin by the beginning: import the modules

In [31]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import scipy.optimize
import statsmodels.stats
import scipy.stats
from tqdm.notebook import tqdm

EPS = np.finfo(np.float).eps

## 1. Task model & Cognitive models

### 1.1 Task definition

**Goal**

Parameters: 
* Number of option ($N$)
* Distribution of probability over these options ($\{p_{reward}(i)\}_{i\in N}$)
* Number of trials ($T$)

**Example: Bandit task**

![title](img/multiarmedbandit.png)

In [32]:
N = 2
P = np.array([0.5, 0.75])
T = 100

# 2. Define your decision-making models: Cognitive model

## Random

* Decision rule:

\begin{equation}
p_{choice}(i) \sim \text{Uniform}
\end{equation} 

* Updating rule

[None]

## Rescola-Wagner RL

* Decision rule ($\tau$)

\begin{equation}
p_{choice}(i) = \dfrac{\exp (v(i)/\tau)}{\sum_{j \in N} \exp (v(j)/\tau)} 
\end{equation}

* Updating rule ($\alpha$)

\begin{equation}
v^{t+1}(i) = v^t(i) + \alpha(s - v^t(i))
\end{equation}

# 3. Simulate 

Determine the (log) likelihood of your model:

\begin{equation}
\log p(D \mid M, \theta) = \sum_{d \in D} \log p(d \mid M, \theta)
\end{equation}


\begin{equation}
\text{BIC} = - 2 \log p(D\mid {\widehat {\theta }},M) + k \log(T)
\end{equation}

In [46]:
def bic(ll, k, n_iteration):
    return -2 *ll + k * np.log(n_iteration)